# Data Validation


- **Need to install python(v3.9.21)**

### Installed packages are :

- anndata - 0.10.3
- jupyter - 1.1.1
- leidenalg - 0.10.1
- matplotlib - 3.9.4
- numpy - 1.26.4
- pandas - 2.2.3
- scanpy - 1.9.6
- seaborn - 0.13.2
- scikit-learn - 1.5.2

In [36]:
data = "pollen"
model = "gatgan"
# k = 90

In [ ]:
import scanpy as sc
import  numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import leidenalg
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import colorsys

/home/bernadettem/anaconda3/envs/env_scRNA2025/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [16]:
from matplotlib import rcParams

# --- Configure Matplotlib & environment ---
# Use a standard LaTeX-compatible font or any known font
rcParams['pdf.fonttype'] = 42  # Use TrueType fonts for editable text in illustrator
rcParams['ps.fonttype'] = 42   # Also for PostScript output
rcParams['font.family'] = 'DejaVu Sans' # A good default font
# %matplotlib inline # This is for Jupyter notebooks, not needed in a .py script

In [17]:
df = pd.read_csv("datafilt1.csv", na_values=['NA'])
df1 = pd.read_csv("datafilt2.csv", na_values=['NA'])
df_comb = pd.read_csv("datafilt_combined.csv", na_values=['NA'])
df.shape

(299, 100)

In [18]:
adata_gen = sc.AnnData(df)
adata_real = sc.AnnData(df1)
adata_comb = sc.AnnData(df_comb)


/home/bernadettem/anaconda3/envs/env_scRNA2025/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/bernadettem/anaconda3/envs/env_scRNA2025/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/bernadettem/anaconda3/envs/env_scRNA2025/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
# label = pd.read_csv(r"/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/real_data/pollenc.txt", header=None, names=["cell_type"])
# label

## FOR CBMC DATA
# label = pd.read_csv(r"/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/real_data/cell_type_cbmc.csv", header = 0, index_col = 0)

## FOR YAN DATA
label = pd.read_csv(r"/path/for/cell/type/data/pollenc.txt", header = None)
label.reset_index(drop=True, inplace=True)
label = label.rename(columns={label.columns[0]: "cell_type"})
label

In [ ]:
label.cell_type.value_counts()

In [21]:
label_encoder = LabelEncoder()     # transform categorical labels into integers

# add one new column into the df3
label['class_label_encoded'] = label_encoder.fit_transform(label['cell_type'])
y = label['class_label_encoded'].values

In [ ]:
set(y)

In [23]:
# from sklearn.preprocessing import LabelEncoder   
# label=pd.read_csv("/vol/eph/data/Data_for_cbmc/cbmcannot.csv")
# label = label.rename(columns={label.columns[0]: "cell_type"})
# class_labels = label.cell_type.value_counts()
# class_labels.values.tolist()
# label_encoder = LabelEncoder()     # transform categorical labels into integers


# # add one new column into the df3
# label['class_label_encoded'] = label_encoder.fit_transform(label['cell_type'])
# y = label['class_label_encoded'].values 

### SAME CODE AS ABOVE - SO DON'T NEEDED

## ARI of Gen Data

In [39]:
def extended_set1_palette(num_colors):
    base_palette = sns.color_palette('Set1', 9)
    if num_colors <= 9:
        return base_palette[:num_colors]
    else:
        # Repeat base palette as many times as needed
        repeats = (num_colors // 9) + 1
        extended_palette = base_palette * repeats
        # Slightly adjust hue to make colors distinct beyond base palette
        def adjust_color_rgb(rgb, hue_shift=0.1):
            h, l, s = colorsys.rgb_to_hls(*rgb)
            h = (h + hue_shift) % 1.0
            return colorsys.hls_to_rgb(h, l, s)

        adjusted_palette = []
        for i in range(num_colors):
            base_color = extended_palette[i]
            # Hue shift depends on which repetition we are in
            rep_shift = (i // 9) * 0.15
            adjusted_color = adjust_color_rgb(base_color, rep_shift)
            adjusted_palette.append(adjusted_color)
        return adjusted_palette

In [ ]:
sc.tl.pca(adata_gen, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata_gen,n_pcs=20,log=True)

In [ ]:
#create neighborhood graph using 20 pcs
sc.pp.neighbors(adata_gen, n_neighbors=20, n_pcs=23)
##dim reduction using umap
sc.tl.umap(adata_gen)

In [ ]:
predicted_labels_gen = adata_gen.obs['leiden'].astype(str).astype(int).to_numpy()

In [4]:
# ari = {}
# i = 0.1
# while(i <= 2.0):
#     sc.tl.leiden(adata_gen, resolution=i)
#     ari_score = adjusted_rand_score(y, adata_gen.obs['leiden'])
#     nmi_score = normalized_mutual_info_score(y, adata_gen.obs['leiden'])
#     macro_f1 = f1_score(y, predicted_labels_gen, average='macro', zero_division=0)
    
    
#     if ari_score > 0.65: ari[i] = [ari_score, nmi_score, macro_f1]
    
#     i += 0.01

In [ ]:
## UMAP with the integer label of the cell-types

sc.tl.leiden(adata_gen, resolution=1.56)
ari_score = adjusted_rand_score(y, adata_gen.obs['leiden'])
nmi_score = normalized_mutual_info_score(y, adata_gen.obs['leiden'])
macro_f1 = f1_score(y, predicted_labels_gen, average='macro', zero_division=0)

# Ensure 'leiden' is categorical
adata_gen.obs['leiden'] = adata_gen.obs['leiden'].astype('category')
num_clusters = len(adata_gen.obs['leiden'].cat.categories)

palette = extended_set1_palette(num_clusters)


sc.pl.umap(adata_gen, color=['leiden'], palette=palette, title=f'UMAP : ARI = {round(ari_score, 4)}, NMI = {round(nmi_score, 4)}, macro_f1 = {round(macro_f1, 4)}', show= False)
# plt.savefig(f'/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/generated_umap/generated_umap_{model}_{data}_k_{k}.png', dpi=300, bbox_inches='tight')

## rasterization

ax = plt.gca()
for artist in ax.collections:
    artist.set_rasterized(True)
    
plt.savefig(
    # f'/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/generated_umap/generated_umap_{model}_{data}_k_{k}_cv2.png',
    f'/path/to/save/the/umap/generated_umap_{model}_{data}_cv2.png',
    dpi=600,
    bbox_inches='tight'
)

In [ ]:
## UMAP with the actual label of the cell-types
int_labels = adata_gen.obs['leiden'].cat.categories.astype(int)
str_labels = label_encoder.inverse_transform(int_labels)
adata_gen.obs['leiden_named'] = adata_gen.obs['leiden'].cat.rename_categories(str_labels)


sc.tl.leiden(adata_gen, resolution=1.56)
ari_score = adjusted_rand_score(y, adata_gen.obs['leiden'])
nmi_score = normalized_mutual_info_score(y, adata_gen.obs['leiden'])
macro_f1 = f1_score(y, predicted_labels_gen, average='macro', zero_division=0)


adata_gen.obs['leiden'] = adata_gen.obs['leiden'].astype('category')
num_clusters = len(adata_gen.obs['leiden'].cat.categories)

palette = extended_set1_palette(num_clusters)

sc.pl.umap(adata_gen, color=['leiden_named'], title=f'UMAP : ARI = {round(ari_score, 4)}, NMI = {round(nmi_score, 4)}, macro_f1 = {round(macro_f1, 4)}', show= False)

ax = plt.gca()
for artist in ax.collections:
    artist.set_rasterized(True)
    
# plt.savefig(
#     f'/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/generated_umap_paper/generated_umap_{model}_{data}_k_{k}_actual_label.pdf',
#     dpi=600,
#     bbox_inches='tight'
# )

plt.savefig(
    # f'/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/generated_umap/generated_umap_{model}_{data}_k_{k}_actual_label_cv2.png',
    f'/path/to/save/the/umap/generated_umap_{model}_{data}_actual_label_cv2.png',
    dpi=600,
    bbox_inches='tight'
)

## ARI of Real Data

In [ ]:
sc.tl.pca(adata_real, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata_real,n_pcs=20,log=True)

In [26]:
#create neighborhood graph using 20 pcs
sc.pp.neighbors(adata_real, n_neighbors=30, n_pcs=17)
##dim reduction using umap
sc.tl.umap(adata_real)

In [28]:
predicted_labels_real = adata_real.obs['leiden'].astype(str).astype(int).to_numpy()

In [3]:
# ari = {}
# i = 0.1
# while(i <= 0.8):
#     sc.tl.leiden(adata_real, resolution=i)
#     ari_score = adjusted_rand_score(y, adata_real.obs['leiden'])
#     nmi_score = normalized_mutual_info_score(y, adata_real.obs['leiden'])
#     macro_f1 = f1_score(y, predicted_labels_real, average='macro', zero_division=0)
    
#     if ari_score < 0.85: ari[i] = [ari_score, nmi_score, macro_f1]
    
#     i += 0.01

In [ ]:
## UMAP with the integer label of the cell-types

sc.tl.leiden(adata_real, resolution=0.18)
ari_score = adjusted_rand_score(y, adata_real.obs['leiden'])
nmi_score = normalized_mutual_info_score(y, adata_real.obs['leiden'])
macro_f1 = f1_score(y, predicted_labels_real, average='macro', zero_division=0)

# Ensure 'leiden' is categorical
adata_real.obs['leiden'] = adata_real.obs['leiden'].astype('category')
num_clusters = len(adata_real.obs['leiden'].cat.categories)

palette = extended_set1_palette(num_clusters)


sc.pl.umap(adata_real, color=['leiden'], palette=palette, title=f'UMAP : ARI = {round(ari_score, 4)}, NMI = {round(nmi_score, 4)}, macro_f1 = {round(macro_f1, 4)}', show= False)
# plt.savefig(f'/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/generated_umap/generated_umap_{model}_{data}_k_{k}.png', dpi=300, bbox_inches='tight')

## rasterization

ax = plt.gca()
for artist in ax.collections:
    artist.set_rasterized(True)
    
plt.savefig(
    f'/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/generated_umap/real_umap_{model}_{data}_cv2.png',
    dpi=600,
    bbox_inches='tight'
)

In [ ]:
## UMAP with the actual label of the cell-types
int_labels = adata_real.obs['leiden'].cat.categories.astype(int)
str_labels = label_encoder.inverse_transform(int_labels)
adata_real.obs['leiden_named'] = adata_real.obs['leiden'].cat.rename_categories(str_labels)


sc.tl.leiden(adata_real, resolution=0.18)
ari_score = adjusted_rand_score(y, adata_real.obs['leiden'])
nmi_score = normalized_mutual_info_score(y, adata_real.obs['leiden'])
macro_f1 = f1_score(y, predicted_labels_real, average='macro', zero_division=0)


adata_real.obs['leiden'] = adata_real.obs['leiden'].astype('category')
num_clusters = len(adata_real.obs['leiden'].cat.categories)

palette = extended_set1_palette(num_clusters)

sc.pl.umap(adata_real, color=['leiden_named'], title=f'UMAP : ARI = {round(ari_score, 4)}, NMI = {round(nmi_score, 4)}, macro_f1 = {round(macro_f1, 4)}', show= False)

ax = plt.gca()
for artist in ax.collections:
    artist.set_rasterized(True)
    
plt.savefig(
    f'/path/to/save/the/umap/real_umap_{model}_{data}_cv2_actual_label.pdf',
    dpi=600,
    bbox_inches='tight'
)

## combined

In [ ]:
sc.tl.pca(adata_comb, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata_comb,n_pcs=20,log=True)

In [ ]:
#create neighborhood graph using 20 pcs
sc.pp.neighbors(adata_comb, n_neighbors=20, n_pcs=30)
##dim reduction using umap
sc.tl.umap(adata_comb)

In [ ]:
predicted_labels_comb = adata_comb.obs['leiden'].astype(str).astype(int).to_numpy()

In [2]:
# ari = {}
# i = 0.0
# while(i <= 2.0):
#     sc.tl.leiden(adata_comb, resolution=i)
#     ari_score = adjusted_rand_score(y, adata_comb.obs['leiden'])
#     nmi_score = normalized_mutual_info_score(y, adata_comb.obs['leiden'])
#     macro_f1 = f1_score(y, predicted_labels_comb, average='macro', zero_division=0)
    
#     if ari_score < 0.9: ari[i] = [ari_score, nmi_score, macro_f1]
    
#     i += 0.01

In [ ]:
## UMAP with the integer label of the cell-types

sc.tl.leiden(adata_comb, resolution=0.34)
ari_score = adjusted_rand_score(y, adata_comb.obs['leiden'])
nmi_score = normalized_mutual_info_score(y, adata_comb.obs['leiden'])
macro_f1 = f1_score(y, predicted_labels_comb, average='macro', zero_division=0)

# Ensure 'leiden' is categorical
adata_comb.obs['leiden'] = adata_comb.obs['leiden'].astype('category')
num_clusters = len(adata_comb.obs['leiden'].cat.categories)

palette = extended_set1_palette(num_clusters)


sc.pl.umap(adata_comb, color=['leiden'], palette=palette, title=f'UMAP : ARI = {round(ari_score, 4)}, NMI = {round(nmi_score, 4)}, macro_f1 = {round(macro_f1, 4)}', show= False)
# plt.savefig(f'/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/generated_umap/generated_umap_{model}_{data}_k_{k}.png', dpi=300, bbox_inches='tight')

## rasterization

ax = plt.gca()
for artist in ax.collections:
    artist.set_rasterized(True)
    
plt.savefig(
    # f'/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/generated_umap/comb_umap_{model}_{data}.png',
    f'/path/to/save/the/umap/comb_umap_{model}_{data}_cv2.png',
    dpi=600,
    bbox_inches='tight'
)

In [ ]:
## UMAP with the actual label of the cell-types
int_labels = adata_comb.obs['leiden'].cat.categories.astype(int)
str_labels = label_encoder.inverse_transform(int_labels)
adata_comb.obs['leiden_named'] = adata_comb.obs['leiden'].cat.rename_categories(str_labels)


sc.tl.leiden(adata_comb, resolution=0.34)
ari_score = adjusted_rand_score(y, adata_comb.obs['leiden'])
nmi_score = normalized_mutual_info_score(y, adata_comb.obs['leiden'])
macro_f1 = f1_score(y, predicted_labels_comb, average='macro', zero_division=0)


adata_comb.obs['leiden'] = adata_comb.obs['leiden'].astype('category')
num_clusters = len(adata_comb.obs['leiden'].cat.categories)

palette = extended_set1_palette(num_clusters)

sc.pl.umap(adata_comb, color=['leiden_named'], title=f'UMAP : ARI = {round(ari_score, 4)}, NMI = {round(nmi_score, 4)}, macro_f1 = {round(macro_f1, 4)}', show= False)

ax = plt.gca()
for artist in ax.collections:
    artist.set_rasterized(True)
    
plt.savefig(
    # f'/home/bernadettem/bernadettenotebook/Ritwik/NLP/GAT_GAN/generated_umap/comb_umap_{model}_{data}_actual_label.png',
    f'/path/to/save/the/umap/comb_umap_{model}_{data}_actual_label_cv2.png',
    dpi=600,
    bbox_inches='tight'
)